In [ ]:
#https://python.langchain.com/docs/integrations/toolkits/openapi

In [13]:
import os, yaml
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec
import spotipy
from dotenv import load_dotenv
from langchain.llms.openai import OpenAI
from langchain.agents.agent_toolkits.openapi import planner
import openai

In [2]:
#Collecting openai and soptify OpenAPI Spec
!wget https://raw.githubusercontent.com/openai/openai-openapi/master/openapi.yaml
!mv openapi.yaml openai_openapi.yaml
!wget https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
!mv openapi.yaml spotify_openapi.yaml

--2023-09-16 17:38:47--  https://raw.githubusercontent.com/openai/openai-openapi/master/openapi.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152811 (149K) [text/plain]
Saving to: 'openapi.yaml'

openapi.yaml        100%[===================>] 149.23K  --.-KB/s    in 0.1s    

2023-09-16 17:38:47 (1.50 MB/s) - 'openapi.yaml' saved [152811/152811]

--2023-09-16 17:38:48--  https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2893

In [3]:
#Getting reducde open api spec needed for efficent processing
with open("openai_openapi.yaml") as f:
    raw_openai_api_spec = yaml.load(f, Loader=yaml.Loader)
openai_api_spec = reduce_openapi_spec(raw_openai_api_spec)

with open("spotify_openapi.yaml") as f:
    raw_spotify_api_spec = yaml.load(f, Loader=yaml.Loader)
spotify_api_spec = reduce_openapi_spec(raw_spotify_api_spec)

In [23]:
#spotfiy credetials
load_dotenv('/Users/kpathak/Library/CloudStorage/OneDrive-ManhattanAssociates/Learnings/api_key.env')
SPOTIPY_CLIENT_ID=os.environ.get('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET=os.environ.get('SPOTIPY_CLIENT_SECRET')
SPOTIPY_REDIRECT_URI=os.environ.get('SPOTIPY_REDIRECT_URI')
print(SPOTIPY_REDIRECT_URI)

http://localhost:8088/callback/


In [5]:
#Getting your credential for your app get from below link
#https://developer.spotify.com/documentation/web-api/concepts/authorization
import spotipy.util as util
from langchain.requests import RequestsWrapper


def construct_spotify_auth_headers(raw_spec: dict):
    scopes = list(
        raw_spec["components"]["securitySchemes"]["oauth_2_0"]["flows"][
            "authorizationCode"
        ]["scopes"].keys()
    )
    access_token = util.prompt_for_user_token(scope=",".join(scopes))
    return {"Authorization": f"Bearer {access_token}"}


# Get API credentials.
headers = construct_spotify_auth_headers(raw_spotify_api_spec)
requests_wrapper = RequestsWrapper(headers=headers)

In [6]:
#Spec length
endpoints = [
    (route, operation)
    for route, operations in raw_spotify_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]
len(endpoints)

63

In [7]:
import tiktoken

enc = tiktoken.encoding_for_model("text-davinci-003")

def count_tokens(s):
    return len(enc.encode(s))


count_tokens(yaml.dump(raw_spotify_api_spec))

81143

In [19]:
#Testing with examples
llm = OpenAI(model_name="gpt-4", temperature=0.0)

In [20]:
#Setting openapi key -
openai.api_key = 'XXXXXXXXXXXXXX'

In [22]:
#Making user query action done through the agent
#Here user wants to create a splotify playlist for song blue

spotify_agent = planner.create_openapi_agent(spotify_api_spec, requests_wrapper, llm)
user_query = (
    "make me a playlist with the first song from kind of blue. call it  Blues."
)
spotify_agent.run(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to create a playlist with the first song from the album "Kind of Blue"
Observation: 1. GET /albums with a query param to search for "Kind of Blue" album
2. GET /albums/{id}/tracks to get the tracks from the "Kind of Blue" album
3. GET /me to find the user's id
4. POST /users/{user_id}/playlists to create a new playlist for the user
5. POST /playlists/{playlist_id}/tracks to add the first song from the "Kind of Blue" album to the newly created playlist
Thought:I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /albums with a query param to search for "Kind of Blue" album
2. GET /albums/{id}/tracks to get the tracks from the "Kind of Blue" album
3. GET /me to find the user's id
4. POST /users/{user_id}/playlists to create a new playlist for the user
5. POST /playlists/{playlist_id}/tracks to add the first song from the "Kind of Blue" album to the newly

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-6sGW5iSU1s0PrTlfnyBArBIn on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-6sGW5iSU1s0PrTlfnyBArBIn on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-6sGW5iSU1s0PrTlfnyBArBIn on tokens per min. Limit: 10000 / min. Please try again in 6ms. C

KeyboardInterrupt: 